In [7]:
#1) Una red social almacena el contenido de los chats entre sus usuarios en un RDD que tiene registros con el siguiente 
#formato: (chat_id, user_id, nickname, text). Queremos saber cuál es el usuario (user_id) que mas preguntas hace en sus 
#chats, contabilizamos una pregunta por cada caracter “?” que aparezca en el campo text. Programar en Spark un programa 
#que identifique al usuario preguntón. (*) (15 pts)

chatsData = [(1, 1, "euge93","what is data science?"), 
             (2, 2, "abrden", "its the science that studies data"),
             (3, 1, "euge93","really?"),
             (4, 3, "czarnia","are you sure?"),
             (5, 1, "euge93","sounds like she is right"),
             (6, 3, "czarnia","oh, allright then"),
             (7, 3, "czarnia","is the exam next week?"),
             (8, 1, "euge93","on Thursday"),
             (9, 3, "czarnia","should we practice PySpark???"),
             (10, 2, "abrden", "yeah"),
            ]
chatsRDD = sc.parallelize(chatsData, 2)


In [18]:
chatsRDD.filter(lambda row: "?" in row[3]).map(lambda row: (row[1], row[3].count("?")))\
.reduceByKey(lambda row1, row2: row1+row2)\
.reduce(lambda row1, row2: row1 if (row1[1] > row2[1]) else row2)[0]

3

In [23]:
# UBER almacena en un cluster todos los datos sobre el movimiento y viajes de todos sus vehículos. 
#Existe un proceso que nos devuelve un RDD llamado trip_summary con los siguientes campos: 
#(driver_id, car_id, trip_id, customer_id, date (YYYYMMDD), distance_traveled), Programar usando Py Spark 
#un programa que nos indique cual fue el conductor con mayor promedio de distancia recorrida por viaje para 
#Abril de 2016. (***) (15 pts)
import datetime

tripsData = [(101202, 100, 234, "pass12", datetime.date(2015, 10, 17),25), 
             (30000, 202, 501, "pass13", datetime.date(2016,4, 18),50),
             (101202, 100, 4000, "pass14", datetime.date(2016, 4, 24),4), 
             (30000, 202, 3500, "pass15", datetime.date(2016, 4, 12),3),
             (101202, 100, 4002, "pass16", datetime.date(2016, 4, 21),10), 
             (12000, 5000, 9000, "pass17", datetime.date(2016, 4, 22),9),
             (12000, 5000, 90300, "pass18", datetime.date(2016, 4, 23),11),
             (30000, 202, 7000, "pass19", datetime.date(2016, 4, 24),8),
             (101202, 100, 9033, "pass20", datetime.date(2016, 4, 24),6), 
             (30000, 202, 8000, "pass21", datetime.date(2016, 4, 24),20)
            ]
trip_summary = sc.parallelize(tripsData, 4)

In [34]:
def avg_dist(row):
    if (row[1][1] == 0): return 0
    return row[1][0] / float(row[1][1])

trip_summary.filter(lambda row: row[4] <= datetime.date(2016,4,30) and row[4] >= datetime.date(2016, 4, 1))\
            .map(lambda row:(row[0], (row[5], 1)))\
            .reduceByKey(lambda row1, row2: (row1[0] + row2[0], row1[1] + row2[1]))\
            .map(lambda row: (avg_dist(row) , row[0])).reduce(lambda row1, row2: row1 if (row1[0] > row2[0]) else row2)
        

(20.25, 30000)

In [21]:
# Tenemos una colección de documentos (textos) almacenados en un cluster. Se quiere establecer un ranking de los patrones
#mas frecuentes para la aparición de letras en las palabras. Por ejemplo “sister”, “ejects” , “ninety” y “amazon” 
#responden al patrón “abacde”. Programar en map-reduce un programa que genere como resultado un listado de tipo 
#(patron, frecuencia) indicando cuántas veces aparece cada patrón en la colección de documentos. Usar combiners 
#como método de agregación.  (**) (15 pts)

docsRDD = sc.parallelize(["My sister is the best singer ever", "A dragons ejects fire through its mouth", 
                          "My grandfather is ninety years old", "I have never purchased anything on amazon"], 2)
LETTERS = ["a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "ñ", "o", "p", "q",
              "r", "s", "t", "u", "v", "w", "x", "y", "z"]

def pattern(word):
    read = {}
    word_pattern = ""
    i = 0
    for c in word:
        if c not in read.keys():
            word_pattern += LETTERS[i]
            read[c] = i
            i += 1
        else:
            word_pattern += LETTERS[read[c]]
    return word_pattern

#Obtengo la lista de pares (patron, frecuencia)
patternList = docsRDD.flatMap(lambda row: [(pattern(word),1) for word in row.split()]).reduceByKey(lambda row1, row2: row1 + row2)
print "TODOS:"
print patternList.collect()

#Obtengo el patron mas frecuente:
mostFrequent = patternList.reduce(lambda row1, row2: row1 if row1[1] > row2[1] else row2)
print "MAS FRECUENTE:"
print mostFrequent

#Obtengo el top five de los pares (patron, frecuencia):
topFive = patternList.takeOrdered(5, lambda row: -row[1])
print "TOP FIVE:" + str(topFive)

orderedAlpha = patternList.sortByKey().collect()
print "ORDENADOS ALFABÉTICAMENE:" + str(orderedAlpha)

inverseRank = patternList.sortBy(lambda row: row[1]).take(5)
print "INVERSE RANK:" + str(inverseRank)

TODOS:
[('abcd', 3), ('a', 2), ('abacde', 4), ('abcdefghi', 1), ('abcde', 2), ('abcdefbg', 1), ('abcdefb', 1), ('abac', 1), ('ab', 5), ('abcbd', 1), ('abcdefcghib', 1), ('abc', 3), ('abcdefg', 1), ('abcdef', 1)]
MAS FRECUENTE:
('ab', 5)
TOP FIVE:[('ab', 5), ('abacde', 4), ('abcd', 3), ('abc', 3), ('a', 2)]
ORDENADOS ALFABÉTICAMENE:[('a', 2), ('ab', 5), ('abac', 1), ('abacde', 4), ('abc', 3), ('abcbd', 1), ('abcd', 3), ('abcde', 2), ('abcdef', 1), ('abcdefb', 1), ('abcdefbg', 1), ('abcdefcghib', 1), ('abcdefg', 1), ('abcdefghi', 1)]
INVERSE RANK:[('abcdefghi', 1), ('abcdefbg', 1), ('abcdefb', 1), ('abac', 1), ('abcbd', 1)]


In [4]:

hpData = ["October arrived, spreading a damp chill over the grounds and into the castle.", 
             "Madam Pomfrey, the nurse, was kept busy by a sudden spate of colds among the staff and students.",
             "Her Pepperup potion worked instantly, though it left the drinker smoking at the ears for several hours afterward",
             "Ginny Weasley, who had been looking pale, was bullied into taking some by Percy",
             "The steam pouring from under her vivid hair gave the impression that her whole head was on fire."
            ]
hpRDD = sc.parallelize(hpData, 2)
#1st approach: supongo que me pide las frases posibles con estas longitudes dentro de los documentos:

def caracteresPorFrase(phrase, lengths):
    character_count_by_length = {}
    words = phrase.split()
    word_count = len(words)
    for phrase_length in lengths:
        i = 0
        j = phrase_length-1
        char_count = 0
        #voy tomando palabras de a grupos de longitud phrase_length
        while (j < word_count):
            #estoy suponiendo frases lindas unidas por 1 único espacio
            phrase = (" ").join([ words[k] for k in xrange(i, j+1) ])
            char_count += len(phrase)
            
            i += 1
            j += 1
        
        #i es el indice inferior de cada frase que tomamos, asi que lo usamos como contador
        #para calcular el promedio de las longitudes que sumamos en el loop
        character_count_by_length[phrase_length] = (char_count, i+1)
    
    return [ (phrase_length, avg) for phrase_length, avg in character_count_by_length.items()]

hpRDD.flatMap(lambda row: caracteresPorFrase(row, [i for i in xrange(2,5)]))\
     .reduceByKey(lambda row1, row2: (row1[0] + row2[0], row1[1] + row2[1]))\
     .map(lambda row: (row[0], row[1][0]/float(row[1][1]))).collect()


[(2, 9.703703703703704), (4, 19.788732394366196), (3, 14.81578947368421)]

In [7]:
#1) Se tiene un archivo con información sobre visitas a páginas web de la forma: (URL, visitas, fecha).
#Existe solo un registro por día para cada URL. Se quiere generar un archivo que, por cada URL, 
#indique cuál fue la fecha en la que tuvo mas visitas y la cantidad de visitas. 
#Programar lo pedido en Map Reduce usando agregación para minimizar la cantidad de datos que deben 
#transferirse en la red. Atención: La resolución es muy simple, trivial, asi que es fundamental resolver 
#la agregación para el puntaje completo. (**) (15 pts)

import datetime

urls = [("www.google.com", 1000, datetime.date(2015, 10, 17)),
          ("www.google.com", 1300, datetime.date(2015, 10, 18)),
          ("www.google.com", 1254, datetime.date(2015, 10, 19)),
          ("www.facebook.com", 1700, datetime.date(2015, 10, 17)),
          ("www.facebook.com", 1100, datetime.date(2015, 10, 18)),
          ("www.facebook.com", 1204, datetime.date(2015, 10, 19)),
          ("www.twitter.com", 1700, datetime.date(2015, 10, 17)),
          ("www.twitter.com", 1100, datetime.date(2015, 10, 18)),
          ("www.twitter.com", 12000, datetime.date(2015, 10, 19)),
          ("www.bing.com", 170, datetime.date(2015, 10, 17)),
          ("www.bing.com", 100, datetime.date(2015, 10, 18)),
          ("www.bing.com", 12, datetime.date(2015, 10, 19))]

urlsRDD = sc.parallelize(urls, 2)

urlsRDD.map(lambda row: (row[0], (row[1], row[2])) ).reduceByKey(lambda row1, row2: row1 if (row1[0] > row2[0]) else row2).collect()


[('www.twitter.com', (12000, datetime.date(2015, 10, 19))),
 ('www.facebook.com', (1700, datetime.date(2015, 10, 17))),
 ('www.bing.com', (170, datetime.date(2015, 10, 17))),
 ('www.google.com', (1300, datetime.date(2015, 10, 18)))]

In [15]:
#Se tiene un archivo de texto con datos de la forma (url_id, user_id) representando visitas a un website. 
#(Se pueden repetir registros). Se pide programar un script que indique cual es el 
#promedio global de sitios visitados por usuario. (***) (15pts)

usersSites= [("user1","www.google.com"),
            ("user1","www.google.com"),
            ("user1","www.facebook.com"),
            ("user1","www.twitter.com"),
            ("user2","www.google.com"),
            ("user2","www.facebook.com"),
            ("user2","www.google.com"),
            ("user2","www.google.com"),
            ("user2","www.despegar.com"),
            ("user3","www.booking.com"),
            ("user3","www.almundo.com"),
            ("user3","www.despegar.com"),
            ("user3","www.trivago.com"),
            ("user3","www.google.com"),
            ("user4","www.google.com"),
            ("user4","www.rae.com"),
            ("user4","www.oxforddictionaries.com"),
            ("user4","www.oxforddictionaries.com"),
            ("user4","www.facebook.com"),
            ("user4","www.twitter.com")]

usersSitesRDD = sc.parallelize(usersSites, 2)

usersSitesRDD.distinct().map(lambda row: (row[0], 1))\
                        .reduceByKey(lambda row1, row2: row1 + row2)\
                        .flatMap(lambda row: [row[1]])\
                        .mean()
            
#Quiero contar cuantos registros distintos tengo:
usersSitesRDD.distinct().count()

usersSitesRDD.countByValue()


defaultdict(int,
            {('user1', 'www.facebook.com'): 1,
             ('user1', 'www.google.com'): 2,
             ('user1', 'www.twitter.com'): 1,
             ('user2', 'www.despegar.com'): 1,
             ('user2', 'www.facebook.com'): 1,
             ('user2', 'www.google.com'): 3,
             ('user3', 'www.almundo.com'): 1,
             ('user3', 'www.booking.com'): 1,
             ('user3', 'www.despegar.com'): 1,
             ('user3', 'www.google.com'): 1,
             ('user3', 'www.trivago.com'): 1,
             ('user4', 'www.facebook.com'): 1,
             ('user4', 'www.google.com'): 1,
             ('user4', 'www.oxforddictionaries.com'): 2,
             ('user4', 'www.rae.com'): 1,
             ('user4', 'www.twitter.com'): 1})

In [106]:
#Un sitio de ventas online registra un log de los productos visitados por los 
#usuarios en un RDD de Spark con registros de la forma (user_id, product_id,date). Existe un registro por cada 
#visita de cada producto. Se quiere obtener un listado de los productos que se han visitado más de 500 veces 
#y para los cuales el día de la semana con mayor cantidad de visitas es el viernes.

#para poder probar vamos a usar cantidad de visitas mínimas = 5

visitedProducts = [("euge123",1,'viernes'), ("euge123",2,'lunes'), ("euge123",3,'sabado'),
                  ("abrden",4,'viernes'), ("abrden",2,'lunes'), ("abrden",3,'sabado'),
                  ("czarnia",4,'viernes'), ("czarnia",2,'martes'), ("czarnia",1,'sabado'),
                  ("euge123",4,'viernes'), ("mporto",3,'jueves'), ("mporto",1,'sabado'),
                  ("slazzari",2,'viernes'), ("slazzari",1,'jueves'), ("slazzari",1,'viernes'),
                  ("abrden",2,'domingo'), ("euge123",1,'jueves'), ("slazzari",1,'viernes'),
                  ("mporto",3,'domingo'), ("mporto",4,'sabado'), ("slazzari",1,'viernes')]

prodsRDD = sc.parallelize(visitedProducts, 2)

def joinDictionaries(row1,row2):
    for day, visit_count in row2.items():
        if row1.has_key(day):
            row1[day] += visit_count
        else:
            row1[day] = visit_count
    return row1

def hasMinimumValueSum(visits_per_day, n_visits):
    count = 0
    for visit_count in visits_per_day.values():
        count += visit_count
    return count > n_visits
    
def hasMaxCountOnKey(visits_per_day, day):
    max_val = visits_per_day[day]
    for week_day in visits_per_day.keys():
        if (day != week_day) and (visits_per_day[week_day] > max_val):
            return False
    return True
    
prodsRDD.map(lambda row: (row[1], {row[2] : 1} ))\
        .reduceByKey(lambda row1, row2: joinDictionaries(row1,row2))\
        .filter(lambda row: hasMinimumValueSum(row[1], 5) and hasMaxCountOnKey(row[1], 'viernes'))\
        .collect()

#Otro enfoque:
#.reduceByKey(lambda row1, row2: (row1[0][x] + row2[0][x]) for x in xrange(0,len(row1)) )
#.filter(lambda row: (row[1][1] > 500) and ( row[1][0][VIERNES] >= x for x in row[1][0]))\

VIERNES = 4
DATES = ["lunes", "martes", "miercoles", "jueves", "viernes", "sabado", "domingo"]
prodsRDD = sc.parallelize(visitedProducts, 2)
prodsRDD.map(lambda row: (row[1], ([ 1 if (row[2] == day) else 0  for day in DATES ] , 1)) )\
        .reduceByKey(lambda row1, row2: ( [row1[0][x] + row2[0][x] for x in xrange(0,len(row1[0]))], row1[1]+ row2[1] ) )\
        .filter(lambda row: (row[1][1] > 5) and ( row[1][0][VIERNES] >= x for x in row[1][0]))\
        .collect()

[(1, ([0, 0, 0, 2, 4, 2, 0], 8))]

In [134]:
# Se tiene un RDD con libros en donde cada registro es un texto.  
#Se pide obtener todos los anagramas de mas de 7 letras que puedan encontrarse.  
#El formato de salida debe ser una lista de listas en donde cada lista tiene un conjunto de palabras que son anagramas.
#Ejemplo: [[discounter,introduces,reductions],[percussion,supersonic]...] (****) (15 pts)

librosData = [("Decir que Inglaterra es el mejor pais es una presuposicion, Alemania lidera la economia mundial" ),
              ("Las fresas le producen alergia, cuando las come delira y dice frases incoherentes"),
              ("La superposicion de papeles dificultaba las cosas"),
              ("Integrarla al mundo sera una tarea dificil"),
              ("Los Enamoramientos se dan armoniosamente en Roma que es la ciudad del Amor"),
              ("Energeticamente entro en la habitacion"),
              ("genericamente")]

#.filter(lambda len_word_list: len_word_list[0] > 7)\
def is_anagram(word1, word2):
    char_list = word1.lower().split()
    for i in xrange(0, len(word2)):
        if word2[i] in char_list:
            char_list.pop(i)
    return char_list == []

librosRDD = sc.parallelize(librosData,2)
librosRDD.flatMap(lambda texto: texto.split())\
         .map(lambda word: (len(word),[word]))\
         .reduceByKey(lambda row1, row2: row1 + row2)\
         .collect()


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 193.0 failed 1 times, most recent failure: Lost task 0.0 in stage 193.0 (TID 324, localhost): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/eugenia/spark-1.6.0-bin-hadoop2.6/python/lib/pyspark.zip/pyspark/worker.py", line 111, in main
    process()
  File "/home/eugenia/spark-1.6.0-bin-hadoop2.6/python/lib/pyspark.zip/pyspark/worker.py", line 106, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/home/eugenia/spark-1.6.0-bin-hadoop2.6/python/pyspark/rdd.py", line 2346, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/home/eugenia/spark-1.6.0-bin-hadoop2.6/python/pyspark/rdd.py", line 2346, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/home/eugenia/spark-1.6.0-b